# Setup & Imports

Mount to drive if needed 

In [ ]:
from google.colab import files
from IPython.display import Image
from google.colab import drive
drive.mount('/gdrive')

Install Tape dependencies 

Github: https://github.com/songlab-cal/tape

In [ ]:
!pip3 install tape_proteins
!pip3 install torch==1.5.1

Imports

In [ ]:
import string
import pandas as pd
import numpy as np
import typing
import copy
import json
import logging
import os
from io import open
import math

import torch
from torch.nn.utils.weight_norm import weight_norm
from torch import nn
import torch.nn.functional as F

from tape import ProteinBertModel, TAPETokenizer

# Download & Load Train Data

The Train Data and Test Data is available here: http://www.cbs.dtu.dk/services/NetSurfP/

In [ ]:
!wget http://www.cbs.dtu.dk/services/NetSurfP-2.0/training_data/Train_HHblits.npz

Generally we don't unzip .npz files. We use ``` np.load() ``` and load it directly to memory. But, the ```data.npy``` file is too large and crashes the Colab instance when trying to load. So, firsr unzip it below and then memory map the file. 

Before unzipping, you can also copy the .npz file to Google Drive using ```!cp $filename $path_to_gdrive``` and then load using ```np.load(path_to_gdrive, mmap_mode='r')```

In [ ]:
!unzip /content/Train_HHblits.npz  ## Zipped file is 420 MB, when unzipped it is 9GB+!

A memory-mapped array is kept on disk. However, it can be accessed and sliced like any ndarray. Memory mapping is especially useful for accessing small fragments of large files without reading the entire file into memory. Syntax: ```np.load(filepath, mmap_mode='r')```

In [ ]:
train_data = np.load('/content/data.npy', mmap_mode='r')
train_labels = np.load('/content/pdbids.npy', mmap_mode='r')  ## 9GB file!

# Breaking down the [train data](http://www.cbs.dtu.dk/services/NetSurfP/)

We have shown for one protein. It is your job to do it for the rest.

The data is represented this way. There are ```10848``` different protein sequences and largest sequence is ```1632``` amino acids long. And, each amino acid has the following data:

```
 [0:20] Amino Acids (sparse encoding)
 Unknown residues are stored as an all-zero vector
 [20:50] hmm profile
 [50] Seq mask (1 = seq, 0 = empty)
 [51] Disordered mask (0 = disordered, 1 = ordered)
 [52] Evaluation mask (For CB513 dataset, 1 = eval, 0 = ignore)
 [53] ASA (isolated)
 [54] ASA (complexed)
 [55] RSA (isolated)
 [56] RSA (complexed)
 [57:65] Q8 GHIBESTC (Q8 -> Q3: HHHEECCC)
 [65:67] Phi+Psi
 [67] ASA_max
```
Read the paper for better understanding. 

In [ ]:
train_data.shape

(10848, 1632, 68)

Converting the Sparse encoded amino acid sequence and the experimentally determined secondary structure to the fasta format. Basically fasta format is a string of single letter codes of amino acids of a protein. So, if a protein is made of 300 amino acids, the fasta format will contain a string of 300 letters representing the amino acids present in the protein.

In [ ]:
code = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
ss = ['H','H','H','E','E','C','C','C']

In [ ]:
protein_0 = train_data[0, :, :]

In [ ]:
length = 0
fasta = ''
for i in range(train_data.shape[1]):
  for j in range(20):
    if(protein_0[i][j]==1):
      fasta = fasta + code[j]
      length+=1

fasta

'MKTAYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQDNLSGAEKAVQVKVKALPDAQFEVVHSLAKWKRQTLGQHDFSAGEGLYTHMKALRPDEDRLSPLHSVYVDQWDWERVMGDGERQFSTLKSTVEAIWAGIKATEAAVSEEFGLAPFLPDQIHFVHSQELLSRYPDLDAKGRERAIAKDLGAVFLVGIGGKLSDGHRHDVRAPDYDDWSTPSELGHAGLNGDILVWNPVLEDAFELSSMGIRVDADTLKHQLALTGDEDRLELEWHQALLRGEMPQTIGGGIGQSRLTMLLLQLPHIGQVQAGVWPAAVRESVPSLL'

In [ ]:
structure = ''
secondary_structure = []
for i in range(train_data.shape[1]):
  for j in range(57,65):
    if(protein_0[i][j]==1):
      structure = structure + ss[j-57]

structure

'CCCCHHHHHHHHHHHHHHHHHHHHHHHCEEECCCCCEEECCCCCCCCCCCCCCCCEECCCCCCCCCEEECCCCCCHHHHHHHHCCCCCCCEEEEEEEEECCCCCCCCCCCCCEEEEEEEEEECCCCCCCHHHHHHHHHHHHHHHHHHHHHHHHHCCCCCCCCCCCEEEEHHHHHHHCCCCCHHHHHHHHHHHHCEEEEECCCCCCCCCCCCCCCCCCCECCCCECCCCCECCEEEEEEEECCCCEEEEEEEEEEECCHHHHHHHHHHHCCCCHHHCHHHHHHHCCCCCCEEEEEEEHHHHHHHHHCCCCHHHCCCCCCCHHHHHHCCCCC'

#Input the sequences into the BERT encoder

In [ ]:
model = ProteinBertModel.from_pretrained('bert-base')
tokenizer = TAPETokenizer(vocab='iupac')  # iupac is the vocab for TAPE models, use unirep for the UniRep model


sequence = fasta
token_ids = torch.tensor([tokenizer.encode(sequence)])
output = model(token_ids)
sequence_output = output[0]
pooled_output = output[1]

In [ ]:
sequence_output.shape

torch.Size([1, 332, 768])

In [ ]:
pooled_output.shape

torch.Size([1, 768])